## 幾種思路

思路1：TF-IDF + 機器學習分類器
直接使用TF-IDF對文本提取特徵，並使用分類器進行分類。在分類器的選擇上，可以使用SVM、LR、或者XGBoost。

思路2：FastText
FastText是入門款的詞向量，利用Facebook提供的FastText工具，可以快速構建出分類器。

思路3：WordVec + 深度學習分類器
WordVec是進階款的詞向量，並通過構建深度學習分類完成分類。深度學習分類的網絡結構可以選擇TextCNN、TextRNN或者BiLSTM。

思路4：Bert詞向量
Bert是高配款的詞向量，具有強大的建模學習能力。

## 獲取6種模型的資料集

In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import re
path = '../data/0414/review_data(seg+pos+stopwords)_n+v+f+p.csv'
df = pd.read_csv(path)

### 檢查重複值、空值

In [2]:
#印出重複資料
print(df[df.duplicated()])

Empty DataFrame
Columns: [reviews, value, comfort, location, cleanliness, service, facilities, ws_pos_reviews, filtered, filtered_word]
Index: []


In [3]:
#移除重複值
#df = df.drop_duplicates()
#print(df.shape)

In [4]:
#印出空值資料
#df_train[df_train.isnull().T.any()]

### 切分為6個資料集

In [5]:
def split_df(df):
    df_value = df[['value','filtered_word']]
    df_value.rename(columns={'value': 'label'}, inplace=True)
    df_comfort = df[['comfort','filtered_word']]
    df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
    df_location = df[['location','filtered_word']]
    df_location.rename(columns={'location': 'label'}, inplace=True)
    df_cleanliness = df[['cleanliness','filtered_word']]
    df_cleanliness.rename(columns={'cleanliness': 'label'}, inplace=True)
    df_service = df[['service','filtered_word']]
    df_service.rename(columns={'service': 'label'}, inplace=True)
    df_facilities = df[['facilities','filtered_word']]
    df_facilities.rename(columns={'facilities': 'label'}, inplace=True)
    return df_value, df_comfort, df_location, df_cleanliness, df_service, df_facilities

In [6]:
#df_value_train, df_comfort_train, df_location_train, df_cleanliness_train, df_service_train ,df_facilities_train = split_df(df_train)
#df_value_test, df_comfort_test, df_location_test, df_cleanliness_test, df_service_test, df_facilities_test = split_df(df_test)
df_value, df_comfort, df_location, df_cleanliness, df_service ,df_facilities = split_df(df)

C:\Users\g3190\AppData\Local\Temp/ipykernel_21928/3099386144.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_value.rename(columns={'value': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_21928/3099386144.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_comfort.rename(columns={'comfort': 'label'}, inplace=True)
C:\Users\g3190\AppData\Local\Temp/ipykernel_21928/3099386144.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [7]:
df_service

,label,filtered_word
0,1.0,"價格(Na),合理(VH),舒適(VH),房間(Nc),老闆娘(Na),人(Na),好(VH..."
1,0.0,"內部(Ncd),房間(Nc),乾淨(VH),場地(Na),團體(Na),使用(VC),覺得(..."
2,0.0,"房間(Nc),小(VH),美中不足(VH),乾(VH),濕(VH),分離(VHC)"
3,0.0,"房子(Na),設計(VC),棒(VH),房間(Nc),採光(Na),好(VH),大廳(Nc)..."
4,1.0,"Cp值(FW),高(VH),乾淨(VH),舒適(VH),空間(Na),大樓(Na),下(Nc..."
...,...,...
1267,1.0,"港式(Na),飲茶(VA),餐廳(Nc),口味(Na),棒(VH),環境(Na),乾淨(VH..."
1268,1.0,"場地(Na),氣派(Na),丁香魚(Na),酥脆(VH),服務(VC),親切(VH),蠟味(..."
1269,1.0,"交通(Na),方便(VH),地下室(Nc),停車場(Nc),良好(VH),菜色(Na),好(..."
1270,1.0,"地點(Na),佳(VH),離(P),逢甲(Nb),夜市(Nc),老闆娘(Na),親切(VH)..."


### 清理資料(移除詞性標註的文字)

In [8]:
def remove_N_comma(sentence):
    # 把後面(N..)(V..)(F..)拿掉
    sentence = str(sentence)
    pattern = re.compile(r"\([N,V,F,P].*?\)") #移除詞性標示
    sentence = re.sub(pattern, '', sentence)
    pattern = re.compile(r",") #將逗號替換為空格
    sentence = re.sub(pattern, ' ', sentence)
    return sentence
pd.options.mode.chained_assignment = None  # 忽略警告

In [9]:
#訓練集
df_service['filtered_word'] = df_service.apply(lambda x: remove_N_comma(x['filtered_word']),axis=1)

In [10]:
print(df_service.shape)
df_service

(1272, 2)


,label,filtered_word
0,1.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,0.0,房間 小 美中不足 乾 濕 分離
3,0.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,1.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,1.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,1.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,1.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,1.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


## 模型架構

### 套件引用

In [11]:
#import package
#轉向量用
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer 
from scipy.sparse import coo_matrix

from imblearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate
import pickle #儲存模型用
from sklearn.model_selection import train_test_split
#類別採樣
import imblearn.over_sampling as over_sampling
import imblearn.under_sampling as under_sampling
import imblearn.combine as combine
from imblearn.pipeline import make_pipeline as make_pipeline_imb


#模型
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
#from xgboost.sklearn import XGBClassifier

#模型效能表現
import sklearn.metrics as metrics

### 顯示訓練結果

In [12]:
def classification_report(y_test, pre):
    #混淆矩陣
    confusion = metrics.confusion_matrix(y_test, pre)
    TP = confusion[1,1]
    TN = confusion[0,0]
    FP = confusion[0,1]
    FN = confusion[1,0]
    print("TP:",TP)
    print("TN:",TN)
    print("FP:",FP)
    print("FN:",FN)
    #Accuracy
    accuracy = (TP+TN)/float(TP+TN+FN+FP)
    print("Accuracy：", accuracy)
    #Sensitivity(Recall)
    recall = TP/float(TP+FN)
    print("Recall：", recall)
    #Specificity
    specificity = TN/float(TN+FP)
    print("Specificity：", specificity)
    #Precision
    precision = TP/float(TP+FP)
    print("Precision：", precision)
    #f1-score
    f1_score = ((2*precision*recall)/(precision+recall))
    print("f1_score：", f1_score)
    #AUC
    print("AUC：", metrics.roc_auc_score(y_test, pre))

In [13]:
df_service

,label,filtered_word
0,1.0,價格 合理 舒適 房間 老闆娘 人 好 做 早餐 旅客 重點 早餐 吃到飽
1,0.0,內部 房間 乾淨 場地 團體 使用 覺得 棒
2,0.0,房間 小 美中不足 乾 濕 分離
3,0.0,房子 設計 棒 房間 採光 好 大廳 挑高 氣派 房價 合理 台東 住 民宿
4,1.0,Cp值 高 乾淨 舒適 空間 大樓 下 免費 吐司 咖啡 老闆 回復 速度
...,...,...
1267,1.0,港式 飲茶 餐廳 口味 棒 環境 乾淨 機車 汽車 停車位 位於 高鐵 附近 適合 宴客
1268,1.0,場地 氣派 丁香魚 酥脆 服務 親切 蠟味 蘿蔔糕 份量 一些 好 牛肉粥 好吃
1269,1.0,交通 方便 地下室 停車場 良好 菜色 好 空間 設計好 說 一流 飯店
1270,1.0,地點 佳 離 逢甲 夜市 老闆娘 親切 服務 房間 舒適 浴室 乾淨


### 切分訓練、測試數據

In [14]:
from sklearn.model_selection import train_test_split
def split_label(df,seed):
    X = df.filtered_word.tolist()
    y = df.label
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=1/3,random_state=seed)
    return X_train,X_test,y_train,y_test

In [15]:
X_train_1,X_test_1,y_train_1,y_test_1 = split_label(df_service,1)
X_train_2,X_test_2,y_train_2,y_test_2 = split_label(df_service,2)
X_train_3,X_test_3,y_train_3,y_test_3 = split_label(df_service,3)

In [16]:
s1 = pd.Series(y_train_1)
freq1 = s1.value_counts() 
print(freq1) 
s2 = pd.Series(y_train_2)
freq2 = s2.value_counts() 
print(freq2) 
s3 = pd.Series(y_train_3)
freq3 = s3.value_counts() 
print(freq3) 

1.0    524
0.0    324
Name: label, dtype: int64
1.0    526
0.0    322
Name: label, dtype: int64
1.0    537
0.0    311
Name: label, dtype: int64


### 模型設計

#### (1) baseline

In [17]:
def SVM_model(X_train,X_test,y_train,y_test):
    print("SVM baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [18]:
def LR_model(X_train,X_test,y_train,y_test):
    print("LR baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), LogisticRegression(random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [19]:
def RF_model(X_train,X_test,y_train,y_test):
    print("RF baseline")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(max_df=0.8,min_df=5,dtype=np.float32), RandomForestClassifier(max_depth=2, random_state=0))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [20]:
def AdaBoost_model(X_train,X_test,y_train,y_test):
    print("AdaBoost_model")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (2)執行採樣 => 解決類別不平衡 (SVM)

In [21]:
def SVM_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [22]:
def SVM_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [23]:
def SVM_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [24]:
def SVM_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), svm.SVC(kernel='linear'))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

#### (3)執行採樣 => 解決類別不平衡 (Adaboost)

In [25]:
def AdaBoost_model2(X_train,X_test,y_train,y_test):
    print("ADASYN")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.ADASYN(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [26]:
def AdaBoost_model3(X_train,X_test,y_train,y_test):
    print("SMOTE")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.SMOTE(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [27]:
def AdaBoost_model4(X_train,X_test,y_train,y_test):
    print("RandomOverSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), over_sampling.RandomOverSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

In [28]:
def AdaBoost_model5(X_train,X_test,y_train,y_test):
    print("RandomUnderSampler")
    #切分數據集
    #X_train,y_train,X_test,y_test = split_label(df_train, df_test)
    #模型架構
    model = make_pipeline_imb(TfidfVectorizer(), under_sampling.RandomUnderSampler(), AdaBoostClassifier(n_estimators=200))
    model.fit(X_train, y_train)
    #模型預測
    pre = model.predict(X_test)
    #麼行評估
    classification_report(y_test, pre)
    print("\n")

## 模型訓練&結果

### 資料集1

#### baseline

In [29]:
SVM_model(X_train_1,X_test_1,y_train_1,y_test_1)
LR_model(X_train_1,X_test_1,y_train_1,y_test_1)
RF_model(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model(X_train_1,X_test_1,y_train_1,y_test_1)

SVM baseline
TP: 237
TN: 148
FP: 7
FN: 32
Accuracy： 0.9080188679245284
Recall： 0.8810408921933085
Specificity： 0.9548387096774194
Precision： 0.9713114754098361
f1_score： 0.9239766081871346
AUC： 0.917939800935364


LR baseline
TP: 247
TN: 142
FP: 13
FN: 22
Accuracy： 0.9174528301886793
Recall： 0.9182156133828996
Specificity： 0.9161290322580645
Precision： 0.95
f1_score： 0.9338374291115311
AUC： 0.917172322820482


RF baseline
TP: 269
TN: 0
FP: 155
FN: 0
Accuracy： 0.6344339622641509
Recall： 1.0
Specificity： 0.0
Precision： 0.6344339622641509
f1_score： 0.7763347763347763
AUC： 0.5


AdaBoost_model
TP: 231
TN: 133
FP: 22
FN: 38
Accuracy： 0.8584905660377359
Recall： 0.8587360594795539
Specificity： 0.8580645161290322
Precision： 0.9130434782608695
f1_score： 0.8850574712643678
AUC： 0.8584002878042931




#### 處理類別不平衡(SVM)

In [30]:
SVM_model2(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model3(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model4(X_train_1,X_test_1,y_train_1,y_test_1)
SVM_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
TP: 236
TN: 149
FP: 6
FN: 33
Accuracy： 0.9080188679245284
Recall： 0.8773234200743495
Specificity： 0.9612903225806452
Precision： 0.9752066115702479
f1_score： 0.923679060665362
AUC： 0.9193068713274973


SMOTE
TP: 235
TN: 146
FP: 9
FN: 34
Accuracy： 0.8985849056603774
Recall： 0.8736059479553904
Specificity： 0.9419354838709677
Precision： 0.9631147540983607
f1_score： 0.9161793372319689
AUC： 0.907770715913179


RandomOverSampler
TP: 235
TN: 148
FP: 7
FN: 34
Accuracy： 0.9033018867924528
Recall： 0.8736059479553904
Specificity： 0.9548387096774194
Precision： 0.9710743801652892
f1_score： 0.9197651663405088
AUC： 0.9142223288164049


RandomUnderSampler
TP: 224
TN: 151
FP: 4
FN: 45
Accuracy： 0.8844339622641509
Recall： 0.8327137546468402
Specificity： 0.9741935483870968
Precision： 0.9824561403508771
f1_score： 0.9014084507042253
AUC： 0.9034536515169684




#### 處理類別不平衡(Adaboost)

In [31]:
AdaBoost_model2(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model3(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model4(X_train_1,X_test_1,y_train_1,y_test_1)
AdaBoost_model5(X_train_1,X_test_1,y_train_1,y_test_1)

ADASYN
TP: 237
TN: 134
FP: 21
FN: 32
Accuracy： 0.875
Recall： 0.8810408921933085
Specificity： 0.864516129032258
Precision： 0.9186046511627907
f1_score： 0.8994307400379505
AUC： 0.8727785106127833


SMOTE
TP: 233
TN: 136
FP: 19
FN: 36
Accuracy： 0.8702830188679245
Recall： 0.8661710037174721
Specificity： 0.8774193548387097
Precision： 0.9246031746031746
f1_score： 0.8944337811900193
AUC： 0.8717951792780909


RandomOverSampler
TP: 236
TN: 134
FP: 21
FN: 33
Accuracy： 0.8726415094339622
Recall： 0.8773234200743495
Specificity： 0.864516129032258
Precision： 0.9182879377431906
f1_score： 0.8973384030418251
AUC： 0.8709197745533038


RandomUnderSampler
TP: 219
TN: 144
FP: 11
FN: 50
Accuracy： 0.8561320754716981
Recall： 0.8141263940520446
Specificity： 0.9290322580645162
Precision： 0.9521739130434783
f1_score： 0.8777555110220441
AUC： 0.8715793260582805




### 資料集2

#### baseline

In [32]:
SVM_model(X_train_2,X_test_2,y_train_2,y_test_2)
LR_model(X_train_2,X_test_2,y_train_2,y_test_2)
RF_model(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model(X_train_2,X_test_2,y_train_2,y_test_2)

SVM baseline
TP: 232
TN: 149
FP: 8
FN: 35
Accuracy： 0.8985849056603774
Recall： 0.8689138576779026
Specificity： 0.9490445859872612
Precision： 0.9666666666666667
f1_score： 0.9151873767258383
AUC： 0.9089792218325818


LR baseline
TP: 239
TN: 133
FP: 24
FN: 28
Accuracy： 0.8773584905660378
Recall： 0.8951310861423221
Specificity： 0.8471337579617835
Precision： 0.908745247148289
f1_score： 0.9018867924528302
AUC： 0.8711324220520528


RF baseline
TP: 267
TN: 0
FP: 157
FN: 0
Accuracy： 0.6297169811320755
Recall： 1.0
Specificity： 0.0
Precision： 0.6297169811320755
f1_score： 0.7727930535455861
AUC： 0.5


AdaBoost_model
TP: 235
TN: 144
FP: 13
FN: 32
Accuracy： 0.8938679245283019
Recall： 0.8801498127340824
Specificity： 0.9171974522292994
Precision： 0.9475806451612904
f1_score： 0.912621359223301
AUC： 0.898673632481691




#### 處理類別不平衡(SVM)

In [33]:
SVM_model2(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model3(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model4(X_train_2,X_test_2,y_train_2,y_test_2)
SVM_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
TP: 242
TN: 142
FP: 15
FN: 25
Accuracy： 0.9056603773584906
Recall： 0.9063670411985019
Specificity： 0.9044585987261147
Precision： 0.9416342412451362
f1_score： 0.9236641221374046
AUC： 0.9054128199623084


SMOTE
TP: 243
TN: 143
FP: 14
FN: 24
Accuracy： 0.910377358490566
Recall： 0.9101123595505618
Specificity： 0.910828025477707
Precision： 0.9455252918287937
f1_score： 0.9274809160305344
AUC： 0.9104701925141344


RandomOverSampler
TP: 242
TN: 142
FP: 15
FN: 25
Accuracy： 0.9056603773584906
Recall： 0.9063670411985019
Specificity： 0.9044585987261147
Precision： 0.9416342412451362
f1_score： 0.9236641221374046
AUC： 0.9054128199623084


RandomUnderSampler
TP: 224
TN: 149
FP: 8
FN: 43
Accuracy： 0.8797169811320755
Recall： 0.8389513108614233
Specificity： 0.9490445859872612
Precision： 0.9655172413793104
f1_score： 0.8977955911823647
AUC： 0.8939979484243422




#### 處理類別不平衡(Adaboost)

In [34]:
AdaBoost_model2(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model3(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model4(X_train_2,X_test_2,y_train_2,y_test_2)
AdaBoost_model5(X_train_2,X_test_2,y_train_2,y_test_2)

ADASYN
TP: 232
TN: 145
FP: 12
FN: 35
Accuracy： 0.8891509433962265
Recall： 0.8689138576779026
Specificity： 0.9235668789808917
Precision： 0.9508196721311475
f1_score： 0.9080234833659491
AUC： 0.8962403683293971


SMOTE
TP: 234
TN: 147
FP: 10
FN: 33
Accuracy： 0.8985849056603774
Recall： 0.8764044943820225
Specificity： 0.9363057324840764
Precision： 0.9590163934426229
f1_score： 0.9158512720156556
AUC： 0.9063551134330494


RandomOverSampler
TP: 230
TN: 145
FP: 12
FN: 37
Accuracy： 0.8844339622641509
Recall： 0.8614232209737828
Specificity： 0.9235668789808917
Precision： 0.9504132231404959
f1_score： 0.9037328094302555
AUC： 0.8924950499773372


RandomUnderSampler
TP: 216
TN: 141
FP: 16
FN: 51
Accuracy： 0.8419811320754716
Recall： 0.8089887640449438
Specificity： 0.8980891719745223
Precision： 0.9310344827586207
f1_score： 0.8657314629258516
AUC： 0.8535389680097331




### 資料集3

#### baseline

In [35]:
SVM_model(X_train_3,X_test_3,y_train_3,y_test_3)
LR_model(X_train_3,X_test_3,y_train_3,y_test_3)
RF_model(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model(X_train_3,X_test_3,y_train_3,y_test_3)

SVM baseline
TP: 225
TN: 148
FP: 20
FN: 31
Accuracy： 0.8797169811320755
Recall： 0.87890625
Specificity： 0.8809523809523809
Precision： 0.9183673469387755
f1_score： 0.8982035928143713
AUC： 0.8799293154761905


LR baseline
TP: 237
TN: 135
FP: 33
FN: 19
Accuracy： 0.8773584905660378
Recall： 0.92578125
Specificity： 0.8035714285714286
Precision： 0.8777777777777778
f1_score： 0.9011406844106464
AUC： 0.8646763392857143


RF baseline
TP: 256
TN: 0
FP: 168
FN: 0
Accuracy： 0.6037735849056604
Recall： 1.0
Specificity： 0.0
Precision： 0.6037735849056604
f1_score： 0.7529411764705882
AUC： 0.5


AdaBoost_model
TP: 233
TN: 144
FP: 24
FN: 23
Accuracy： 0.8891509433962265
Recall： 0.91015625
Specificity： 0.8571428571428571
Precision： 0.9066147859922179
f1_score： 0.9083820662768032
AUC： 0.8836495535714286




#### 處理類別不平衡(SVM)

In [36]:
SVM_model2(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model3(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model4(X_train_3,X_test_3,y_train_3,y_test_3)
SVM_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
TP: 231
TN: 153
FP: 15
FN: 25
Accuracy： 0.9056603773584906
Recall： 0.90234375
Specificity： 0.9107142857142857
Precision： 0.9390243902439024
f1_score： 0.9203187250996016
AUC： 0.9065290178571428


SMOTE
TP: 231
TN: 152
FP: 16
FN: 25
Accuracy： 0.9033018867924528
Recall： 0.90234375
Specificity： 0.9047619047619048
Precision： 0.9352226720647774
f1_score： 0.9184890656063619
AUC： 0.9035528273809524


RandomOverSampler
TP: 231
TN: 152
FP: 16
FN: 25
Accuracy： 0.9033018867924528
Recall： 0.90234375
Specificity： 0.9047619047619048
Precision： 0.9352226720647774
f1_score： 0.9184890656063619
AUC： 0.9035528273809524


RandomUnderSampler
TP: 218
TN: 161
FP: 7
FN: 38
Accuracy： 0.8938679245283019
Recall： 0.8515625
Specificity： 0.9583333333333334
Precision： 0.9688888888888889
f1_score： 0.9064449064449064
AUC： 0.9049479166666666




#### 處理類別不平衡(Adaboost)

In [37]:
AdaBoost_model2(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model3(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model4(X_train_3,X_test_3,y_train_3,y_test_3)
AdaBoost_model5(X_train_3,X_test_3,y_train_3,y_test_3)

ADASYN
TP: 230
TN: 144
FP: 24
FN: 26
Accuracy： 0.8820754716981132
Recall： 0.8984375
Specificity： 0.8571428571428571
Precision： 0.905511811023622
f1_score： 0.9019607843137255
AUC： 0.8777901785714286


SMOTE
TP: 224
TN: 143
FP: 25
FN: 32
Accuracy： 0.8655660377358491
Recall： 0.875
Specificity： 0.8511904761904762
Precision： 0.8995983935742972
f1_score： 0.8871287128712871
AUC： 0.863095238095238


RandomOverSampler
TP: 224
TN: 145
FP: 23
FN: 32
Accuracy： 0.8702830188679245
Recall： 0.875
Specificity： 0.8630952380952381
Precision： 0.9068825910931174
f1_score： 0.8906560636182903
AUC： 0.8690476190476192


RandomUnderSampler
TP: 229
TN: 147
FP: 21
FN: 27
Accuracy： 0.8867924528301887
Recall： 0.89453125
Specificity： 0.875
Precision： 0.916
f1_score： 0.9051383399209487
AUC： 0.884765625


